In [179]:
import pandas as pd
import numpy as np
import re

In [180]:
data=pd.read_csv('https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore.csv')
df=data
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


### Operazioni preliminari sul dataset googleplaystore

Rimozione di una riga problematica

In [181]:
data[data['Installs']=='Free']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [182]:
data = data.drop(data[data['Installs']=='Free'].index)

Trasformazione dei valori “Varies with device” in missing su tutto il dataset **(Punto 3)**

In [183]:
data[data.isin(['Varies with device'])].head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varies with device,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
data=data.replace('Varies with device',np.nan)

In [185]:
#Check
data[data.isin(['Varies with device'])].head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1)Convert the app sizes to a number

In [186]:
set(data['Size'])

{nan,
 '902k',
 '94M',
 '691k',
 '404k',
 '173k',
 '203k',
 '14M',
 '270k',
 '696k',
 '458k',
 '253k',
 '704k',
 '228k',
 '837k',
 '414k',
 '12M',
 '66M',
 '164k',
 '192k',
 '860k',
 '498k',
 '73M',
 '25M',
 '55M',
 '191k',
 '730k',
 '545k',
 '655k',
 '60M',
 '25k',
 '308k',
 '122k',
 '411k',
 '91M',
 '323k',
 '874k',
 '865k',
 '269k',
 '598k',
 '717k',
 '5.0M',
 '2.4M',
 '914k',
 '378k',
 '32M',
 '88M',
 '41k',
 '80M',
 '23k',
 '5.4M',
 '8.1M',
 '72M',
 '7.6M',
 '220k',
 '11M',
 '784k',
 '153k',
 '961k',
 '695k',
 '916k',
 '551k',
 '8.5k',
 '609k',
 '7.5M',
 '209k',
 '549k',
 '246k',
 '77M',
 '176k',
 '4.1M',
 '642k',
 '3.4M',
 '601k',
 '485k',
 '54k',
 '5.2M',
 '28M',
 '857k',
 '8.4M',
 '9.2M',
 '11k',
 '210k',
 '239k',
 '842k',
 '4.9M',
 '9.7M',
 '7.9M',
 '98M',
 '1.5M',
 '600k',
 '79M',
 '5.3M',
 '93M',
 '266k',
 '853k',
 '714k',
 '18M',
 '318k',
 '314k',
 '801k',
 '400k',
 '2.6M',
 '91k',
 '506k',
 '9.9M',
 '663k',
 '22M',
 '982k',
 '442k',
 '417k',
 '746k',
 '994k',
 '240k',
 '47

In [187]:
#regex
p=re.compile('(?P<cifre>\d+[.]*\d+)(?P<unità>\w)')

def letter_to_number(letter):
    if letter=='M':
        return 1000000
    if letter=='k':
        return 1000
    return 1

#funzione per la conversione dei valori di Size
def conversion(string):
    if(pd.isnull(string)):
        return string
    m=p.match(string)
    l=m.group('unità')
    number=letter_to_number(l)
    finalnumber=float(m.group('cifre'))*number
    intnumber=int(finalnumber)
    return intnumber

Check:

In [188]:
conversion('12.1M')

12100000

Rimozione dei missing della colonna size:

In [189]:
full=len(data)
missing=len(data)-len(data.dropna(subset=['Size']))
print(round((missing/full),2)) # i missing di size costituiscono il 16% del dataset

0.16


In [190]:
data.dropna(subset=['Size'],inplace=True)

Conversione dei valori di Size in quelli desiderati:

In [191]:
data['Size'] = data['Size'].apply(conversion)

In [192]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


### 2- Convert the number of installs to a number

In [193]:
#tutti i valori della colonna da convertire
set(data['Installs'])

{'0+',
 '1+',
 '1,000+',
 '1,000,000+',
 '1,000,000,000+',
 '10+',
 '10,000+',
 '10,000,000+',
 '100+',
 '100,000+',
 '100,000,000+',
 '5+',
 '5,000+',
 '5,000,000+',
 '50+',
 '50,000+',
 '50,000,000+',
 '500+',
 '500,000+',
 '500,000,000+'}

Rimuovo i segni '+' e le virgole, trasformo i valori di Installs in numerici

In [194]:
data['Installs']=data['Installs'].str.replace('[+,]','') 
data['Installs']=data['Installs'].apply(lambda x:int(x))
set(data['Installs'])

{0,
 1,
 5,
 10,
 50,
 100,
 500,
 1000,
 5000,
 10000,
 50000,
 100000,
 500000,
 1000000,
 5000000,
 10000000,
 50000000,
 100000000,
 500000000,
 1000000000}

In [195]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


### 4) Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [196]:
set(df['Current Ver'])

{nan,
 '1.2.3.1',
 '1.2.36',
 '12',
 '4.81',
 '1.131.2',
 '1.2.21',
 '1.17.0.0',
 '18.4.8.0',
 '0.7.0',
 '3.9.3',
 '3.36',
 '4.4.3',
 '1.0',
 '2018-04-27',
 '2.6.158',
 '4.2.2',
 '1.2.20180117',
 '4.4.4',
 '3.2.12',
 '6',
 '2.24',
 '4.6.30',
 '1.0.51.0.3',
 '3.49',
 '1.10.0.user.pro.release',
 '3.6.3',
 '6.6.28',
 '6.2-sayc',
 '2.05.0010',
 '8.0.1.0',
 '4.0.30',
 'Version:6.97',
 '1.12.0.1654',
 '1.13.108869',
 '1.10.5',
 '1.2.0.9',
 '2018-03',
 '3.2.1c',
 '1.23.1.2',
 '1.2.14',
 '5.064',
 '5.403',
 '1.1.9371',
 '3.17.2',
 '2.20 Build 02',
 'Android 2.0 - 2016',
 '1.48.0',
 '1.18.2',
 '7.6.5',
 '1.9.1.180419172630.4a15962',
 '1.21',
 '11.6.0',
 '1.9.0',
 '7.0.0',
 '10.23.2-964159b',
 '5.600.49',
 'v2018.Apr.12.16',
 '3.3.9',
 '3.5.2.2',
 '6.9.2',
 '2.28.1',
 '3.0.188',
 '39.231',
 '1.16.9',
 '6.2.7',
 '2.04.2714',
 '9.1',
 '2.13',
 '1.0.13',
 '1.0.0.22',
 '2.9.7',
 '5.39.1',
 '6.3.3',
 '2.2.15',
 'SolarCT 1.9 v',
 '2.9.2',
 '1.2.1009.695',
 '3.2.994',
 '5.5.8',
 '3.6.4',
 '2.2.00',
 '4

In [197]:
set(data['Android Ver'])

{nan,
 '4.4W and up',
 '4.1 and up',
 '5.1 and up',
 '6.0 and up',
 '2.3.3 and up',
 '3.1 and up',
 '2.2 and up',
 '8.0 and up',
 '2.0 and up',
 '7.0 and up',
 '2.2 - 7.1.1',
 '3.0 and up',
 '5.0 - 8.0',
 '4.4 and up',
 '3.2 and up',
 '4.2 and up',
 '4.1 - 7.1.1',
 '1.6 and up',
 '1.5 and up',
 '5.0 - 7.1.1',
 '2.1 and up',
 '1.0 and up',
 '5.0 - 6.0',
 '4.0.3 - 7.1.1',
 '5.0 and up',
 '4.0 and up',
 '4.3 and up',
 '7.0 - 7.1.1',
 '2.3 and up',
 '4.0.3 and up',
 '7.1 and up',
 '2.0.1 and up'}

In [198]:
#regex per current version
cv = re.compile('(?P<version>(\d{1,4}\.){0,4}(\d){1,4})')
#funzione per la conversione di Current Version
def con_version(string,comp):
    if(pd.isnull(string)):
        return string
    m=comp.search(string)
    if not m:
        return np.nan
    version = m.group('version')
    return version

In [199]:
#regex per android version
av = re.compile('(?P<version>(\d\.){0,2}(\d))\s*\-*\s*(?P<version2>(\d\.){0,2}(\d))*')
#funzione per la conversione di Android Version: nel caso esistano due versioni, seleziona quella più recente
def con_version2(string,comp):
    if(pd.isnull(string)):
        return string
    m=comp.search(string)
    if not m:
        return np.nan
    if m.group('version2'):
        version = m.group('version2')
    else:
        version = m.group('version')
    return version

In [200]:
#Check su un valore di Android version
con_version2('4.0.3 - 7.1.1',av)

'7.1.1'

Applicazione delle precedenti funzioni su Current Ver e Android Ver:

In [201]:
data['Current Ver'] = data['Current Ver'].apply(lambda row: con_version(row,cv))

In [202]:
data['Android Ver'] = data['Android Ver'].apply(lambda row: con_version2(row,av))

In [203]:
set(data['Current Ver'])

{nan,
 '1.2.3.1',
 '1.2.36',
 '12',
 '4.81',
 '1.131.2',
 '1.2.21',
 '1.17.0.0',
 '2.7.11.6',
 '18.4.8.0',
 '0.7.0',
 '3.9.3',
 '3.36',
 '4.4.3',
 '1.0',
 '2.6.158',
 '4.2.2',
 '4.4.4',
 '3.2.12',
 '2750',
 '6',
 '2.24',
 '4.6.30',
 '1.0.51.0.3',
 '6.6.28',
 '3.6.3',
 '2.05.0010',
 '4.0.30',
 '1.12.0.1654',
 '1.2.0.9',
 '1.10.5',
 '1.23.1.2',
 '1.2.14',
 '5.064',
 '5.403',
 '1.1.9371',
 '3.17.2',
 '3.200.1828',
 '1.48.0',
 '1.18.2',
 '7.6.5',
 '1.21',
 '11.6.0',
 '1.8.2031',
 '1.9.0',
 '7.0.0',
 '5.600.49',
 '3.3.9',
 '3.5.2.2',
 '6.9.2',
 '2.28.1',
 '3.0.188',
 '39.231',
 '1.16.9',
 '2.7.11.16',
 '6.2.7',
 '2.04.2714',
 '9.1',
 '2.13',
 '1.0.13',
 '1.0.0.22',
 '2.9.7',
 '5.39.1',
 '6.3.3',
 '2.2.15',
 '1.2.1009.695',
 '2.9.2',
 '3.2.994',
 '5.5.8',
 '3.6.4',
 '2.2.00',
 '4.19.0.2320',
 '1.1.0.0.1487',
 '1.3',
 '6.0.1',
 '3.3.1.41',
 '1.6.1',
 '1.00',
 '1.2.22',
 '2.493',
 '2.29',
 '8.2.9',
 '1.06',
 '6.3.0',
 '6.0',
 '0.1.5',
 '8.2.0',
 '0.1.1009',
 '4.31.2',
 '3.0.0',
 '25.7',
 '2.0.

In [204]:
set(data['Android Ver'])

{nan,
 '5.0',
 '2.1',
 '2.0',
 '7.1.1',
 '1.0',
 '7.1',
 '1.5',
 '2.3.3',
 '3.2',
 '3.0',
 '4.0.3',
 '3.1',
 '4.2',
 '5.1',
 '4.3',
 '2.0.1',
 '4.4',
 '4.1',
 '1.6',
 '7.0',
 '8.0',
 '4.0',
 '6.0',
 '2.2',
 '2.3'}

In [205]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4


### 5) Remove the duplicates

In [206]:
len(data)

9145

In [207]:
data.drop_duplicates(inplace=True)

In [208]:
len(data)

8831

Dal seguente groupby emergono duplicati: righe con stessa app e stessa categoria

In [209]:
dup=data.groupby(['App','Category']).size().reset_index()
dup.columns=['App','Category','Size']
dup[dup['Size']>2]

,App,Category,Size
89,8 Ball Pool,GAME,6
601,Amazon Shopping,SHOPPING,3
642,Angry Birds Classic,GAME,5
649,Angry Birds Rio,GAME,4
1377,BeautyPlus - Easy Photo Editor & Selfie Camera,PHOTOGRAPHY,4
1505,Block Puzzle,GAME,3
1622,Bubble Shooter,GAME,5
2136,Camera360: Selfie Photo Editor with Funny Sticker,PHOTOGRAPHY,3
2152,Candy Crush Saga,GAME,4
2153,Candy Crush Soda Saga,GAME,3


In [210]:
data[data['App']=='Toca Kitchen 2']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2037,Toca Kitchen 2,FAMILY,4.5,1014822,56000000,50000000,Free,0,Everyone,Educational;Pretend Play,"June 29, 2017",1.2.3,4.1
2127,Toca Kitchen 2,FAMILY,4.5,1014846,56000000,50000000,Free,0,Everyone,Educational;Pretend Play,"June 29, 2017",1.2.3,4.1
3920,Toca Kitchen 2,FAMILY,4.5,1013465,56000000,50000000,Free,0,Everyone,Educational;Pretend Play,"June 29, 2017",1.2.3,4.1


Elimino i duplicati sulle colonne App e Category, mantenendo le osservazioni con la review più alta:

In [211]:
data = data.sort_values('Reviews', ascending=False).drop_duplicates(['App', 'Category'],keep='first').sort_index()

Check:

In [212]:
data[data['App']=='Toca Kitchen 2']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2127,Toca Kitchen 2,FAMILY,4.5,1014846,56000000,50000000,Free,0,Everyone,Educational;Pretend Play,"June 29, 2017",1.2.3,4.1


In [213]:
data.sort_index(inplace=True)
len(data)

8495

### 6) For each category, compute the number of apps

In [214]:
data.groupby('Category').size()

Category
ART_AND_DESIGN           62
AUTO_AND_VEHICLES        75
BEAUTY                   47
BOOKS_AND_REFERENCE     194
BUSINESS                376
COMICS                   49
COMMUNICATION           243
DATING                  159
EDUCATION                89
ENTERTAINMENT            64
EVENTS                   57
FAMILY                 1781
FINANCE                 299
FOOD_AND_DRINK           88
GAME                    874
HEALTH_AND_FITNESS      232
HOUSE_AND_HOME           62
LIBRARIES_AND_DEMO       81
LIFESTYLE               334
MAPS_AND_NAVIGATION     107
MEDICAL                 369
NEWS_AND_MAGAZINES      201
PARENTING                54
PERSONALIZATION         351
PHOTOGRAPHY             222
PRODUCTIVITY            291
SHOPPING                168
SOCIAL                  189
SPORTS                  287
TOOLS                   733
TRAVEL_AND_LOCAL        173
VIDEO_PLAYERS           128
WEATHER                  56
dtype: int64

### 7) For each category, compute the average rating

In [215]:
round(data.groupby('Category')['Rating'].mean(),3)

Category
ART_AND_DESIGN         4.361
AUTO_AND_VEHICLES      4.148
BEAUTY                 4.292
BOOKS_AND_REFERENCE    4.323
BUSINESS               4.096
COMICS                 4.168
COMMUNICATION          4.077
DATING                 3.964
EDUCATION              4.374
ENTERTAINMENT          4.155
EVENTS                 4.479
FAMILY                 4.187
FINANCE                4.105
FOOD_AND_DRINK         4.108
GAME                   4.236
HEALTH_AND_FITNESS     4.191
HOUSE_AND_HOME         4.128
LIBRARIES_AND_DEMO     4.203
LIFESTYLE              4.090
MAPS_AND_NAVIGATION    4.009
MEDICAL                4.165
NEWS_AND_MAGAZINES     4.144
PARENTING              4.348
PERSONALIZATION        4.325
PHOTOGRAPHY            4.114
PRODUCTIVITY           4.133
SHOPPING               4.214
SOCIAL                 4.258
SPORTS                 4.204
TOOLS                  4.007
TRAVEL_AND_LOCAL       4.011
VIDEO_PLAYERS          4.022
WEATHER                4.242
Name: Rating, dtype: float64

### 8) Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [216]:
set(data.Genres)

{'Action',
 'Action;Action & Adventure',
 'Adventure',
 'Adventure;Action & Adventure',
 'Adventure;Brain Games',
 'Adventure;Education',
 'Arcade',
 'Arcade;Action & Adventure',
 'Arcade;Pretend Play',
 'Art & Design',
 'Art & Design;Action & Adventure',
 'Art & Design;Creativity',
 'Art & Design;Pretend Play',
 'Auto & Vehicles',
 'Beauty',
 'Board',
 'Board;Action & Adventure',
 'Board;Brain Games',
 'Board;Pretend Play',
 'Books & Reference',
 'Books & Reference;Creativity',
 'Books & Reference;Education',
 'Business',
 'Card',
 'Card;Action & Adventure',
 'Card;Brain Games',
 'Casino',
 'Casual',
 'Casual;Action & Adventure',
 'Casual;Brain Games',
 'Casual;Creativity',
 'Casual;Education',
 'Casual;Music & Video',
 'Casual;Pretend Play',
 'Comics',
 'Comics;Creativity',
 'Communication',
 'Dating',
 'Education',
 'Education;Action & Adventure',
 'Education;Brain Games',
 'Education;Creativity',
 'Education;Education',
 'Education;Music & Video',
 'Education;Pretend Play',
 'Educa

Creo il dataset Genres, con valori della colonna Genres splittati in due differenti colonne.
Considero anche la colonna Rating, utile per analisi successive

In [217]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4


In [218]:
Genres=data[['App','Genres','Rating']]
Genres.head()

,App,Genres,Rating
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design,4.1
1,Coloring book moana,Art & Design;Pretend Play,3.9
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design,4.7
3,Sketch - Draw & Paint,Art & Design,4.5
4,Pixel Draw - Number Art Coloring Book,Art & Design;Creativity,4.3


In [219]:
genre1=Genres['Genres'].str.split(';',expand=True)[0]
genre2=Genres['Genres'].str.split(';',expand=True)[1]

In [220]:
Genres_final=pd.concat([Genres,genre1,genre2],axis=1)
Genres_final.columns=['App','Genres','Rating','Genre1','Genre2']
Genres_final.head()

,App,Genres,Rating,Genre1,Genre2
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design,4.1,Art & Design,None
1,Coloring book moana,Art & Design;Pretend Play,3.9,Art & Design,Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design,4.7,Art & Design,None
3,Sketch - Draw & Paint,Art & Design,4.5,Art & Design,None
4,Pixel Draw - Number Art Coloring Book,Art & Design;Creativity,4.3,Art & Design,Creativity


In [221]:
Genres_final.drop('Genres', axis=1,inplace=True)

Successivamente creo il dataset bridiging:

In [222]:
#seleziono tutte le app con Genre2 (il secondo genere di appartenenza) non nullo
Genres2=Genres_final[['App','Rating','Genre2']]
Genres2=Genres2[Genres2['Genre2'].notnull()]
Genres2.head()

,App,Rating,Genre2
1,Coloring book moana,3.9,Pretend Play
4,Pixel Draw - Number Art Coloring Book,4.3,Creativity
9,Kids Paint Free - Drawing Fun,4.7,Creativity
23,Mcqueen Coloring pages,NaN,Action & Adventure
26,Colorfit - Drawing & Coloring,4.7,Creativity


In [223]:
#creo nuovi indici: dal 10500 in poi, per 370 osservazioni di Genres2
idx=list(range(10500,10500+370))
Genres2.index=idx

In [224]:
#creo un nuovo dataset escludendo la colonna Genre2
Genres1=Genres_final[['App','Rating','Genre1']]
#Rinomino le colonne dei dataset per il concat
Genres1.columns = ['App','Rating','Genre']
Genres2.columns = ['App','Rating','Genre']
bridging=pd.concat([Genres1,Genres2])

In [225]:
#Check:
bridging[bridging['App']=='Pixel Draw - Number Art Coloring Book']

,App,Rating,Genre
4,Pixel Draw - Number Art Coloring Book,4.3,Art & Design
10501,Pixel Draw - Number Art Coloring Book,4.3,Creativity


In [226]:
len(bridging)
#il dataset include 370 osservazioni (che rappresentano le osservazioni con più generi) in più rispetto a quello originale.

8865

Genero, partendo dal dataset genres_final, un dataset contenente solo i generi:

In [227]:
Genres=Genres_final.drop(['App','Rating'], axis=1)
Genres.head()

,Genre1,Genre2
0,Art & Design,None
1,Art & Design,Pretend Play
2,Art & Design,None
3,Art & Design,None
4,Art & Design,Creativity


### 9) For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [228]:
bools=data['Genres'].str.get_dummies(';')
bools.head()

,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Brain Games,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
data=pd.concat([data,bools],axis=1)
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,...,0,0,0,0,0,0,0,0,0,0


In [230]:
#facciamo un check sulla riga 4, che ha 2 Genres: 'Art & Design' e 'Creativity'
data.loc[4]

App                        Pixel Draw - Number Art Coloring Book
Category                                          ART_AND_DESIGN
Rating                                                       4.3
Reviews                                                      967
Size                                                     2800000
Installs                                                  100000
Type                                                        Free
Price                                                          0
Content Rating                                          Everyone
Genres                                   Art & Design;Creativity
Last Updated                                       June 20, 2018
Current Ver                                                  1.1
Android Ver                                                  4.4
Action                                                         0
Action & Adventure                                             0
Adventure                

### 10) For each genre, compute the average rating. What is the genre with highest average?

In [232]:
round(bridging.groupby('Genre')['Rating'].mean(),3)

Genre
Action                     4.238
Action & Adventure         4.285
Adventure                  4.193
Arcade                     4.276
Art & Design               4.359
Auto & Vehicles            4.148
Beauty                     4.292
Board                      4.294
Books & Reference          4.321
Brain Games                4.359
Business                   4.096
Card                       4.051
Casino                     4.269
Casual                     4.143
Comics                     4.168
Communication              4.077
Creativity                 4.335
Dating                     3.964
Education                  4.295
Educational                4.092
Entertainment              4.093
Events                     4.479
Finance                    4.105
Food & Drink               4.108
Health & Fitness           4.192
House & Home               4.128
Libraries & Demo           4.203
Lifestyle                  4.090
Maps & Navigation          4.009
Medical                    4.165
Musi

In [242]:
bridging.groupby('Genre')['Rating'].mean().idxmax()

'Events'

### 11) For each app, compute the approximate income, obtain as a product of number of installs and price

In [234]:
set(data['Price'])

{'$0.99',
 '$1.00',
 '$1.04',
 '$1.20',
 '$1.26',
 '$1.29',
 '$1.49',
 '$1.50',
 '$1.59',
 '$1.61',
 '$1.70',
 '$1.75',
 '$1.76',
 '$1.96',
 '$1.97',
 '$1.99',
 '$10.00',
 '$10.99',
 '$109.99',
 '$11.99',
 '$12.99',
 '$13.99',
 '$14.00',
 '$14.99',
 '$15.46',
 '$15.99',
 '$154.99',
 '$16.99',
 '$17.99',
 '$18.99',
 '$19.40',
 '$19.90',
 '$19.99',
 '$2.00',
 '$2.49',
 '$2.56',
 '$2.59',
 '$2.60',
 '$2.90',
 '$2.99',
 '$200.00',
 '$24.99',
 '$25.99',
 '$28.99',
 '$29.99',
 '$299.99',
 '$3.02',
 '$3.04',
 '$3.08',
 '$3.28',
 '$3.49',
 '$3.61',
 '$3.88',
 '$3.99',
 '$30.99',
 '$33.99',
 '$37.99',
 '$379.99',
 '$389.99',
 '$39.99',
 '$394.99',
 '$399.99',
 '$4.29',
 '$4.49',
 '$4.60',
 '$4.77',
 '$4.80',
 '$4.84',
 '$4.85',
 '$4.99',
 '$400.00',
 '$46.99',
 '$5.00',
 '$5.49',
 '$5.99',
 '$6.49',
 '$6.99',
 '$7.49',
 '$7.99',
 '$74.99',
 '$79.99',
 '$8.49',
 '$8.99',
 '$89.99',
 '$9.00',
 '$9.99',
 '0'}

Rimuovo i segni del dollaro, poi trasformo in float:

In [235]:
data['Price']=data['Price'].str.replace('[$]','') 
data['Price']=data['Price'].apply(lambda x:float(x))
set(data['Price'])

{0.0,
 0.99,
 1.0,
 1.04,
 1.2,
 1.26,
 1.29,
 1.49,
 1.5,
 1.59,
 1.61,
 1.7,
 1.75,
 1.76,
 1.96,
 1.97,
 1.99,
 2.0,
 2.49,
 2.56,
 2.59,
 2.6,
 2.9,
 2.99,
 3.02,
 3.04,
 3.08,
 3.28,
 3.49,
 3.61,
 3.88,
 3.99,
 4.29,
 4.49,
 4.6,
 4.77,
 4.8,
 4.84,
 4.85,
 4.99,
 5.0,
 5.49,
 5.99,
 6.49,
 6.99,
 7.49,
 7.99,
 8.49,
 8.99,
 9.0,
 9.99,
 10.0,
 10.99,
 11.99,
 12.99,
 13.99,
 14.0,
 14.99,
 15.46,
 15.99,
 16.99,
 17.99,
 18.99,
 19.4,
 19.9,
 19.99,
 24.99,
 25.99,
 28.99,
 29.99,
 30.99,
 33.99,
 37.99,
 39.99,
 46.99,
 74.99,
 79.99,
 89.99,
 109.99,
 154.99,
 200.0,
 299.99,
 379.99,
 389.99,
 394.99,
 399.99,
 400.0}

In [236]:
data['Approximate Income']=round(data['Installs']*data['Price'],1)
set(data['Approximate Income'])

{0.0,
 1.0,
 1.5,
 2.0,
 2.5,
 3.0,
 4.0,
 5.0,
 6.0,
 7.4,
 9.9,
 10.0,
 14.9,
 19.6,
 19.9,
 20.0,
 24.9,
 29.0,
 29.9,
 30.0,
 30.4,
 34.9,
 36.1,
 45.0,
 48.0,
 49.5,
 49.9,
 52.0,
 54.9,
 59.9,
 64.9,
 74.5,
 87.5,
 88.0,
 99.0,
 99.5,
 100.0,
 110.0,
 126.0,
 130.0,
 149.0,
 149.5,
 149.9,
 199.0,
 199.5,
 199.9,
 200.0,
 242.5,
 249.0,
 249.5,
 259.9,
 299.0,
 299.5,
 299.9,
 302.0,
 349.0,
 388.0,
 399.0,
 449.0,
 449.5,
 469.9,
 495.0,
 499.0,
 499.5,
 500.0,
 599.0,
 649.0,
 699.0,
 745.0,
 749.5,
 799.0,
 899.9,
 990.0,
 995.0,
 999.0,
 1000.0,
 1245.0,
 1290.0,
 1299.0,
 1400.0,
 1490.0,
 1495.0,
 1500.0,
 1610.0,
 1700.0,
 1745.0,
 1990.0,
 1995.0,
 2245.0,
 2490.0,
 2495.0,
 2499.0,
 2560.0,
 2990.0,
 2995.0,
 3080.0,
 3490.0,
 3990.0,
 3999.0,
 4290.0,
 4490.0,
 4495.0,
 4840.0,
 4950.0,
 4990.0,
 5000.0,
 5990.0,
 6990.0,
 7450.0,
 7490.0,
 7990.0,
 7995.0,
 8490.0,
 8995.0,
 9900.0,
 9950.0,
 9990.0,
 10000.0,
 10990.0,
 12000.0,
 12450.0,
 14950.0,
 14990.0,
 15460.0,

In [237]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word,Approximate Income
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0.0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,0,0,0,0,0,0,0,0,0,0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0.0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0.0,Teen,Art & Design,...,0,0,0,0,0,0,0,0,0,0.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0.0,Everyone,Art & Design;Creativity,...,0,0,0,0,0,0,0,0,0,0.0


### 12) For each app, compute its minimum and maximum Sentiment_polarity

In [238]:
reviews=pd.read_csv('https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore_user_reviews.csv')

In [239]:
reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [240]:
#Valori massimi
reviews.groupby('App')['Sentiment_Polarity'].max().sort_values(ascending=False)

App
10 Best Foods for You                                 1.0
FBReader: Favorite Book Reader                        1.0
Eve Period Tracker - Love, Sex & Relationships App    1.0
BigOven Recipes, Meal Planner, Grocery List & More    1.0
Evie Launcher                                         1.0
ExDialer - Dialer & Contacts                          1.0
Expedia Hotels, Flights & Car Rental Travel Deals     1.0
Bible                                                 1.0
BeyondMenu Food Delivery                              1.0
Betterment                                            1.0
Extreme Racing 2 - Real driving RC cars game!         1.0
EyeEm - Camera & Photo Filter                         1.0
EzCalculator                                          1.0
BestCam Selfie-selfie, beauty camera, photo editor    1.0
Blood Pressure Log - MyDiary                          1.0
Best Wallpapers QHD                                   1.0
FOSSIL Q: DESIGN YOUR DIAL                            1.0
FOX       

In [241]:
#Valori minimi
reviews.groupby('App')['Sentiment_Polarity'].min().sort_values()

App
BestCam Selfie-selfie, beauty camera, photo editor   -1.0
BuzzFeed: News, Tasty, Quizzes                       -1.0
Extreme Match                                        -1.0
Expedia Hotels, Flights & Car Rental Travel Deals    -1.0
Eat Fit - Diet and Health Free                       -1.0
Anthem Anywhere                                      -1.0
Anthem BC Anywhere                                   -1.0
Easy Origami Ideas                                   -1.0
Easy Makeup Tutorials                                -1.0
Easy Installer - Apps On SD                          -1.0
Apartment, Home Rental Search: Realtor.com Rentals   -1.0
DuraSpeed                                            -1.0
Dungeon Hunter Champions: Epic Online Action RPG     -1.0
DroidAdmin for Android - Advice                      -1.0
BÁO MỚI - Đọc Báo, Tin Tức 24h                       -1.0
CAIXA                                                -1.0
Dr. Panda Restaurant 3                               -1.0
Doodle Jum